In [7]:
from benchmark.benchmarkFunctions import StybliskiTang
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from typing import List


def gradient(model:Pipeline, X:List[float], delta_h:float=1e-5):
    """
    Gradient function via finite difference
    """
    # Create an array of zeros to allocate memory
    grad = np.zeros(len(X))
    
    for i in range(len(X)):
        Xh = X.copy() 
        # Add delta H in the i-th dimension
        Xh[i] += delta_h

        # Estimate the gradient via finite difference
        grad[i] = (model.predict(Xh.T) - model.predict(X.T)) / delta_h
    
    return grad.reshape(-1,1)

def BFGS(fun, x0=np.array, x1=np.array, low=-5, high=5, max_iter=100, tol=10e-6):

    # Initialization of the surrogate model
    model = Pipeline([('poly',   PolynomialFeatures(degree=2)), ('linear', LinearRegression(fit_intercept=False))]) 


    # Reshape of the initial points as column vector (Algebra theory)
    x0 = x0.reshape(-1, 1)
    x1 = x1.reshape(-1, 1)

    dim = len(x0)
    B_inv = np.eye(dim)

    x_k    = x0.copy()
    x_next = x1.copy()

    X_log = np.vstack((x_k.T, x_next.T))
    Y_log = np.array(fun(X_log))
    
    for iter in range(max_iter):
        
        # Fit the model on those few samples
        model.fit(X_log, Y_log)

        # Compute the two factors: differnce in the gradient, difference in the position
        x_delta = x_next - x_k
        y       = gradient(model,x_next) - gradient(model, x_k)

        # Update the Inverse of the Hessian approximation
        B_inv = (np.eye(dim) - (x_delta @ y.T)/ (y.T @ x_delta)) @ B_inv @ (np.eye(dim) - (y @ x_delta.T)/(y.T @ x_delta)) + (x_delta @ x_delta.T)/(y.T @ x_delta)
       
        # Update for next iteration
        x_next, x_k = x_k - B_inv @ gradient(model,x_k), x_next.copy()



        X_log = np.vstack((X_log, x_next.T))
        Y_log = np.vstack((Y_log, fun(x_next.T)))

        if (np.linalg.norm(gradient(model, x_next)) < tol):
            print(f"Stopped at iteration {iter}")
            break
        print(iter)
    return X_log, Y_log

f = lambda x : np.array([x.T[0]**2 + x.T[1]**2]).T


x0 = np.array([[-2-1]], dtype=float)
x1 = np.array([[-1,-1]], dtype=float)

X, Y = BFGS(StybliskiTang, x0, x1 , max_iter=1000)

for i in range(len(X)):
    print(X[i], Y[i])







ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 2